In [585]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

%matplotlib inline


In [713]:
def create_cpt(var_names, probs, levels_list):
    """
    This method generates a pandas dataframe as a conditional probability table
    
    Args:
        :param var_names: (list string) list of variable names
        :param probs: (list double) vector of probabilities for the flattened probability table
        :param levels_list: (list string tuples) list of outcomes for each variable i.e. high or low
        
    Returns:
        :return cpt: (pandas.DatFrame) The conditional probability table
    """
    
    # Create the data frame
    cpt = pd.DataFrame(columns=["probs"] + var_names)
    
    # Fill in the probabilities
    cpt["probs"] = probs
    
    # Step through and set the levels of each variable
    levels_table = list(itertools.product(*(levels_list)))
    
    if not len(levels_table) == len(probs):
        print "ERROR:: Incorrect probability dimensions"
        return None
    
    cpt[var_names] = levels_table
    return cpt

def observe(network, var_name, observation):
    """
    Observes a given variable in our network.  This just deletes all entries not consistent
    with the observation
    
    Args:
        :param network: (list pandas.dataframe) tlist of the cpts in the network
        :param var_name: (list string) name of the observed variables
        :param observation: (list string) values that are observed 
    
    Returns:
        :return: (pandas.dataframe) 
    
    """
    observed_network = []
    for cpt in network:
        for var,obs in zip(var_name, observation):
            if var in cpt.columns:
                cpt = cpt[cpt[var] == obs]
        observed_network.append(cpt)
    
    return observed_network

def create_cpt_from_data():
    pass

def marginalize_factor(cpt, factor):
    """
    This method marginalizes out a single factor by summing over all possible value combinations for that factor
    """
    # Drop the factor to be marginalized out
    probs = cpt["probs"]
    cpt = cpt.drop([factor, "probs"], axis=1)
    
    # Create a new table to store marginalized values
    marginalized_cpt = pd.DataFrame(columns=list(cpt.columns))
    marginalized_probs = []
    
    # Marginalize the cpt
    while cpt.shape[0] > 0:
        # Find all positions that have the same feature pattern
        positions = [x for x in range(0, cpt.shape[0]) if sum(cpt.iloc[0] == cpt.iloc[x]) == cpt.shape[1]]
        
        # Sum up all probabilities
        marginalized_probs.append(sum(probs[probs.index[positions]]))
        
        # add the factor configuration to the marginalized cpt
        marginalized_cpt = marginalized_cpt.append(cpt[:1])
        
        # Drop all positions that have been summed
        cpt = cpt.drop(cpt.index[positions], axis=0)
        probs = probs.drop(probs.index[positions], axis=0)
    
    #marginalized_cpt["probs"] = marginalized_probs
    marginalized_cpt.insert(0, "probs", marginalized_probs)
    return marginalized_cpt
        

def marginalize_factors(cpt, factors):
    """
    marginalizes out a list of factors from a single cpt
    
    Args:
        :param cpt: (pandas.DataFrame)
        :param factors: (list string)
    
    Returns:
        :return cpt: (pandas.DataFrame)
    """
    for factor in factors:
        cpt = marginalize_factor(cpt, factor)
    
    return cpt

def marginalize(network, factors):
    """
    marginalizes out a list of factors from a network of cpts
    
    Args:
        :param network: (list pandas.DataFrame)
        :param factors: (list string)
    
    Returns:
        :return marginalized_network: (list pandas.DataFrame)
    """
    marginalized_network = []
    for cpt in network:
        cpt = marginalize_factors(cpt, factors)
        marginalized_network.append(cpt)
    return marginalized_network
        


def factor_product(cpt1, cpt2):
    """
    This method takes the factor product of two cpts by multiplying where the variables overlap
    
    Args:
        :param cpt1: (pandas.DataFrame)
        :param cpt2: (pandas.DataFrame)
        
    Returns:
        :returns cpt: (pandas.DataFrame)
    """
    # Find the names of all columns
    column_names = list(set(cpt1.columns).union(set(cpt2.columns)).difference(set(["probs"])))
    shared_features = list(set(cpt1.columns).intersection(set(cpt2.columns)).difference(set(["probs"])))
    probs = []
    
    # Create an empty data frame to store the product cpt table
    cpt = pd.DataFrame(columns=column_names)
    idx = 0
    for i in range(0, cpt1.shape[0]):
        for j in range(0, cpt2.shape[0]):
            if sum(cpt1[shared_features].iloc[i] == cpt2[shared_features].iloc[j]) == len(shared_features):
                probs.append(cpt1["probs"].iloc[i] * cpt2["probs"].iloc[j])
                vals = []
                for name in column_names:
                    if name in cpt1.columns:
                        vals.append(cpt1[name].iloc[i])
                    else:
                        vals.append(cpt2[name].iloc[j])
                cpt.loc[idx] = vals
                idx = idx + 1
                
    cpt.insert(0, "probs", probs)
    return cpt

def infer(network, marg_vars, obs_vars, obs_vals):
    """
    This function performs inference on a bayesian network
    
    Args:
        :param network: (list pandas.DataFrame)
        :param marg_vars: (list string)
        :param obs_vars: (list string)
        :param obs_vals: (list int)
    
    Returns:
        :returns:
    """
    # Observe variables
    network = observe(network, obs_vars, obs_vals)
    
    # while there are still variables to be marginalized
    for marg_var in marg_vars:
        marg_net = []
        rm_nodes = []
        for i, node in enumerate(network):
            if marg_var in node.columns:
                marg_net.append(node)
                rm_nodes.append(i)
                
        # Check to see if marg_vars is present
        if not len(marg_net) == 0:
            table = marg_net[0]
            
            # delete marginalized nodes from network
            rm_nodes.reverse()
            for rm in rm_nodes:
                del network[rm]
                
            # marginalize out variables
            for cpt in marg_net[1:]:
                table = factor_product(table, cpt)
                
            marginalized_table = marginalize_factor(table, marg_var)
            network.append(marginalized_table)            
    
    # When all variables have been marginalized product the table together
    product = network[0]
    for node in network[1:]:
        product = factor_product(product, node)
        
    # Normalize
    product["probs"] = product["probs"] / sum(product["probs"])
    
    return product
    

In [645]:
var_names = ["x"]
probs = [.3, .7]
levels = [(1, 0)]
x = create_cpt(var_names, probs, levels)
var_names = ["y", "x"]
probs = [.8, .4, .2, .6]
levels = [(1, 0), (1, 0)]
y = create_cpt(var_names, probs, levels)
var_names = ["z", "y"]
probs = [.5, .6, .5, .4]
levels = [(1, 0), (1, 0)]
z = create_cpt(var_names, probs, levels)
var_names = ["x", "y", "z"]
probs = [.5, .6, .5, .4, .1, .2,.3,.4]
levels = [(1, 0), (1, 0), (1, 0)]
w = create_cpt(var_names, probs, levels)
xyznet = [x, y, z]
#cpt = cpt[cpt["x"] != 0]
cpt2 = observe([y], ["x", "y"], [0, 1])
cpt2

[   probs  y  x
 1    0.4  1  0]

In [636]:
infer(xyznet, ["x"], [], [])

True
[   probs  y  x
0    0.8  1  1
1    0.4  1  0
2    0.2  0  1
3    0.6  0  0,    probs  z  y
0    0.5  1  1
1    0.6  1  0
2    0.5  0  1
3    0.4  0  0]
False


[   probs  y  x
 0    0.8  1  1
 1    0.4  1  0
 2    0.2  0  1
 3    0.6  0  0,    probs  z  y
 0    0.5  1  1
 1    0.6  1  0
 2    0.5  0  1
 3    0.4  0  0]

In [588]:
factor_product(x,y)

,probs,y,x
0,0.24,1,1
1,0.06,0,1
2,0.28,1,0
3,0.42,0,0


In [589]:
factor_product(factor_product(x, y), z)

,probs,y,x,z
0,0.120,1,1,1
1,0.120,1,1,0
2,0.036,0,1,1
3,0.024,0,1,0
4,0.140,1,0,1
5,0.140,1,0,0
6,0.252,0,0,1
7,0.168,0,0,0


In [590]:
marginalize_factor(factor_product(x, y), "x")

,probs,y
0,0.52,1
1,0.48,0


In [591]:
marginalize_factor(factor_product(y, z), "z")

,probs,y,x
0,0.8,1,1
2,0.4,1,0
4,0.2,0,1
6,0.6,0,0


In [592]:
observe(xyznet, ["x"], [1])

[   probs  x
 0    0.3  1,    probs  y  x
 0    0.8  1  1
 2    0.2  0  1,    probs  z  y
 0    0.5  1  1
 1    0.6  1  0
 2    0.5  0  1
 3    0.4  0  0]

In [593]:
observe(xyznet, ["x", "y"], [1, 1])

[   probs  x
 0    0.3  1,    probs  y  x
 0    0.8  1  1,    probs  z  y
 0    0.5  1  1
 2    0.5  0  1]

In [595]:
# Koeller Example
var_names = ["a", "b", "c"]
probs = [.25, .35, .08, .16, .05, .07, 0, 0, .15, .21, .09, .18]
levels = [(1, 2, 3), (1, 2), (1, 2)]
koeller = create_cpt(var_names, probs, levels)

In [596]:
marginalize_factor(koeller, "b")

,probs,a,c
0,0.33,1,1
1,0.51,1,2
4,0.05,2,1
5,0.07,2,2
8,0.24,3,1
9,0.39,3,2


In [602]:
# My Example Marginalization
var_names = ["s", "c"]
probs = [.8, .2, .4, .6]
levels = [(1, 0), (1, 0)]
chol = create_cpt(var_names, probs, levels)
var_names = ["s"]
probs = [.15, .85]
levels = [(1, 0)]
smoke = create_cpt(var_names, probs, levels)

In [603]:
marginalize_factor(mine, "c")

,probs,s
0,1,1
2,1,0


In [604]:
factor_product(chol, smoke)

,probs,c,s
0,0.12,1,1
1,0.03,0,1
2,0.34,1,0
3,0.51,0,0


In [637]:
# Bishop Example
b = create_cpt(["battery"], [.9, .1], [(1,0)])
f = create_cpt(["fuel"], [.9, .1], [(1,0)])
g = create_cpt(["gauge", "battery", "fuel"], [.8, .2, .2, .1, .2, .8, .8, .9], [(1,0), (1,0), (1,0)])
car_net = [b, f, g]

In [638]:
g

,probs,gauge,battery,fuel
0,0.8,1,1,1
1,0.2,1,1,0
2,0.2,1,0,1
3,0.1,1,0,0
4,0.2,0,1,1
5,0.8,0,1,0
6,0.8,0,0,1
7,0.9,0,0,0


In [615]:
factor_product(factor_product(b, f), g)

,probs,fuel,battery,gauge
0,0.648,1,1,1
1,0.162,1,1,0
2,0.018,0,1,1
3,0.072,0,1,0
4,0.018,1,0,1
5,0.072,1,0,0
6,0.001,0,0,1
7,0.009,0,0,0


In [616]:
factor_product(factor_product(g, f), b)

,probs,battery,fuel,gauge
0,0.648,1,1,1
1,0.018,1,0,1
2,0.018,0,1,1
3,0.001,0,0,1
4,0.162,1,1,0
5,0.072,1,0,0
6,0.072,0,1,0
7,0.009,0,0,0


In [617]:
marginalize_factor(factor_product(g, b), "gauge")


,probs,fuel,battery
0,0.9,1,1
1,0.9,0,1
2,0.1,1,0
3,0.1,0,0


In [618]:
factor_product(marginalize_factor(g, "gauge"), b)

,probs,battery,fuel
0,0.9,1,1
1,0.9,1,0
2,0.1,0,1
3,0.1,0,0


In [714]:
infer(car_net, ["battery"], ["gauge"], [0])

,probs,fuel,gauge
0,0.742857,1,0
1,0.257143,0,0


In [716]:
infer(car_net, ["battery", "fuel"], [], [])

,probs,gauge
0,0.685,1
1,0.315,0


In [717]:
infer(car_net, ["battery"], ["fuel"], [0])

,probs,fuel,gauge
0,0.19,0,1
1,0.81,0,0


In [718]:
infer(car_net, [],["gauge", "battery"], [0, 0])

,probs,fuel,battery,gauge
0,0.888889,1,0,0
1,0.111111,0,0,0


In [732]:

chol = create_cpt(["s", "c"], [.8, .2, .4, .6], [(1, 0), (1, 0)])
smoke = create_cpt(["s"], [.15, .85], [(1, 0)])
bp = create_cpt(["e", "s", "b"], 
                [.45, .55, .05, .95, .95, .05, .55, .45], 
                [(1, 0), (1, 0), (1, 0)])
exercise = create_cpt(["e"], [.4, .6], [(1, 0)])
attack = create_cpt(["b", "a"], [.75, .25, .05, .95], [(1, 0), (1, 0)])
heart_net = [exercise, smoke, chol, bp, attack]


In [733]:
heart_net

[   probs  e
 0    0.4  1
 1    0.6  0,    probs  s
 0   0.15  1
 1   0.85  0,    probs  s  c
 0    0.8  1  1
 1    0.2  1  0
 2    0.4  0  1
 3    0.6  0  0,    probs  e  s  b
 0   0.45  1  1  1
 1   0.55  1  1  0
 2   0.05  1  0  1
 3   0.95  1  0  0
 4   0.95  0  1  1
 5   0.05  0  1  0
 6   0.55  0  0  1
 7   0.45  0  0  0,    probs  b  a
 0   0.75  1  1
 1   0.25  1  0
 2   0.05  0  1
 3   0.95  0  0]

In [734]:
infer(heart_net, ["c", "b", "e", "s"], ["s"], [1])

,probs,a
0,0.575,1
1,0.425,0


In [735]:
infer(heart_net, ["c", "a", "e", "s"], ["a"], [1])

,probs,b
0,0.912463,1
1,0.087537,0


In [740]:
infer(heart_net, ["c", "a", "b", "e"], ["b"], [1])

,probs,s
0,0.27439,1
1,0.72561,0


In [741]:
infer(heart_net, ["c", "a", "b", "s"], ["b"], [1])

,probs,e
0,0.107317,1
1,0.892683,0


In [744]:
print infer(heart_net, ["c", "a", "b", "e"], ["b", "e"], [1, 1])

      probs  s
0  0.613636  1
1  0.386364  0


In [745]:
infer(heart_net, ["c", "a", "b"], ["b"], [1])

,probs,s,e
0,0.065854,1,1
1,0.041463,0,1
2,0.208537,1,0
3,0.684146,0,0
